In [27]:
import pandas as pd
import re

In [28]:
# Load the original Excel file from Azure
original_df = pd.read_excel("P:/Temp/2.29.2_1_table_0.xlsx")

In [29]:
# Identify the column with Part Numbers using regex
part_number_column = None
part_number_pattern = re.compile(r'P\d{2}-\d{3}-\d{3}')

for column in original_df.columns:
    if original_df[column].astype(str).str.match(part_number_pattern).any():
        part_number_column = column
        break

if part_number_column is None:
    raise ValueError("Part number column not found.")

# Print all the part numbers from the identified part number column
print(original_df[part_number_column].unique())


['P39-031-373' 'P39-031-362' 'P29-071-348' 'P39-052-270' 'P39-052-280'
 'P39-032-215']


In [30]:
original_df.head(5)

,item,profile#,description,dimensions,unit qty.,use,Part Number,unit,pu_quant,pu_price,amount
0,0,9S356,"CASING, COL, WHITE, PAINTED - DECOSMART ON LDF","14MM x 2.1/4"" x 7' = 9/16""",300/unit,5000 SER STD CASE,P39-031-373,PC,1296,4.030,5222.88
1,0,03892-F6192,"CASING, WHITE, PAINTED - DECOSMART ON LDF","5/8"" x 2"" x 16'",250/unit,SHELF TRIM,P39-031-362,LF,4000,0.720,2880.00
2,0,03846 - 82.1/4 (E1),"JAMB, WHITE, PAINTED - DECOSMART ON LDF\n:unse...","15MM x 3.5/8"" = 5/8"" x 82.1/4""",250/unit,"3.5/8"" STD JAMB",P29-071-348,PC,250,7.070,1767.50
3,0,92625,"BASE, WHITE, PAINTED - DECOSMART ON LDF","12MM x 2.1/4"""" = 1/2"" × 16*",200/unit,400 SER BASE,P39-052-270,LF,2512,0.564,1416.77
4,:unselected:,90623 Ogee Base,"BASE, WHITE, PAINTED - DECOSMART ON LDF","12MM x 3.1/4"" = 1/2"""" x 16'",200/unit,5000 SER BASE - 2wk lead,P39-052-280,LF,2512,0.714,1793.57


In [31]:
# Print the headers of the dataframe
print(original_df.columns.tolist())


['item', 'profile#', 'description', 'dimensions', 'unit qty.', 'use', 'Part Number', 'unit', 'pu_quant', 'pu_price', 'amount']


In [ ]:
original_df['pr_codenum'] = original_df['pr_codenum'].str.strip()

In [ ]:
# Load the LPC Excel file
lpc_df = pd.read_excel(
    r"C:\Users\daniel.pace\Documents\Coding\Purchasing Automation\PA_V4\Price Comparison\Last Part Cost 3.1.xlsx",
    sheet_name='All')


po_cost = lpc_df['PO Cost']

po_cost.head(5)

0      NaN
1     6.24
2     8.61
3    44.50
4      NaN
Name: PO Cost, dtype: float64

In [ ]:
print(lpc_df.columns.tolist())

['Facility', 'Part Number', 'Description', 'IM Factor', 'IM UOM', 'Standard Cost', 'Standard Cost Update', 'Last Valid PO', 'PO Number', 'Po Received', 'PO Cost', 'PO Factor', 'PO UOM', 'PO Converted Cost', 'Change In Cost', 'CM?']


In [33]:
# Map PO Cost from LPC to original dataframe based on part numbers
original_df = original_df.merge(lpc_df[['Part Number', 'PO Cost']], on='Part Number', how='left')

original_df.head(5)

,item,profile#,description,dimensions,unit qty.,use,Part Number,unit,pu_quant,pu_price,amount,PO Cost
0,0,9S356,"CASING, COL, WHITE, PAINTED - DECOSMART ON LDF","14MM x 2.1/4"" x 7' = 9/16""",300/unit,5000 SER STD CASE,P39-031-373,PC,1296,4.030,5222.88,4.030
1,0,03892-F6192,"CASING, WHITE, PAINTED - DECOSMART ON LDF","5/8"" x 2"" x 16'",250/unit,SHELF TRIM,P39-031-362,LF,4000,0.720,2880.00,11.520
2,0,03846 - 82.1/4 (E1),"JAMB, WHITE, PAINTED - DECOSMART ON LDF\n:unse...","15MM x 3.5/8"" = 5/8"" x 82.1/4""",250/unit,"3.5/8"" STD JAMB",P29-071-348,PC,250,7.070,1767.50,7.070
3,0,92625,"BASE, WHITE, PAINTED - DECOSMART ON LDF","12MM x 2.1/4"""" = 1/2"" × 16*",200/unit,400 SER BASE,P39-052-270,LF,2512,0.564,1416.77,9.024
4,:unselected:,90623 Ogee Base,"BASE, WHITE, PAINTED - DECOSMART ON LDF","12MM x 3.1/4"" = 1/2"""" x 16'",200/unit,5000 SER BASE - 2wk lead,P39-052-280,LF,2512,0.714,1793.57,11.424


In [34]:
# Save the updated DataFrame to an Excel file
original_df.to_excel('P:/Temp/2.29.2_1_table_0.xlsx', index=False)


In [38]:
# Assuming df is your DataFrame
# Add a new column based on the condition
original_df['New Column'] = original_df.apply(lambda row: round(row['PO Cost'], 2) if row['pu_price'] == row['PO Cost'] else round(row['amount']/row['PO Cost'], 2), axis=1)
original_df.head(5)

,item,profile#,description,dimensions,unit qty.,use,Part Number,unit,pu_quant,pu_price,amount,PO Cost,New Column
0,0,9S356,"CASING, COL, WHITE, PAINTED - DECOSMART ON LDF","14MM x 2.1/4"" x 7' = 9/16""",300/unit,5000 SER STD CASE,P39-031-373,PC,1296,4.030,5222.88,4.030,4.03
1,0,03892-F6192,"CASING, WHITE, PAINTED - DECOSMART ON LDF","5/8"" x 2"" x 16'",250/unit,SHELF TRIM,P39-031-362,LF,4000,0.720,2880.00,11.520,250.00
2,0,03846 - 82.1/4 (E1),"JAMB, WHITE, PAINTED - DECOSMART ON LDF\n:unse...","15MM x 3.5/8"" = 5/8"" x 82.1/4""",250/unit,"3.5/8"" STD JAMB",P29-071-348,PC,250,7.070,1767.50,7.070,7.07
3,0,92625,"BASE, WHITE, PAINTED - DECOSMART ON LDF","12MM x 2.1/4"""" = 1/2"" × 16*",200/unit,400 SER BASE,P39-052-270,LF,2512,0.564,1416.77,9.024,157.00
4,:unselected:,90623 Ogee Base,"BASE, WHITE, PAINTED - DECOSMART ON LDF","12MM x 3.1/4"" = 1/2"""" x 16'",200/unit,5000 SER BASE - 2wk lead,P39-052-280,LF,2512,0.714,1793.57,11.424,157.00
